In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

batch_size = 64
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\raw


Extracting ./data/MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\raw


Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\raw


Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\raw
Processing...


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20,kernel_size = 5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320,10)
        
    def forward(self,x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [10]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # Print
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [11]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output,target,size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
for epoch in range(1, 10):
    train(epoch)
    test()

<ipython-input-9-e3c1890c2b60>:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.285294
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.279042
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.277351
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.259419
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.241486
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.216723
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.197504
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.110595
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.140122
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.979715
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.843805
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.606514
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.578077
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.463610
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.872565
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.023983
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.883096
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.718657
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.934109
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.672786
Train Epoch: 1 [12800/60000 (

<ipython-input-11-cc97db07b8d6>:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1954, Accuracy: 9413/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.195949
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.173348
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.232606
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.152473
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.258605
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.194659
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.128710
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.159732
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.066851
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.191604
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.162369
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.087417
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.143323
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.237566
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.143929
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.163775
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.091624
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.111756
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.092124
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.066605
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.140580
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.053627
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.227640
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.186078
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.144697
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.151883
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.110464
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.135733
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.063586
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.097422
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.029812
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.208849
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.088398
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.151191
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.021496
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.168291
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.030567
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.098071
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.027415


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.026552
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.128326
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.011698
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.157103
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.050908
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.193922
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.095409
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.075054
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.070538
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.117943
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.115400
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.097289
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.119120
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.091702
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.069929
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.078108
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.060898
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.110866
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.050197
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.215114


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.057371
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.056189
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.025718
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.012109
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.016590
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.087887
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.012825
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.009518
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.053956
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.057784
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.049727
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.017721
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.188256
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.043680
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.097242
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.041235
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.044617
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.094998
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.044339
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.018943


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.026453
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.138910
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.117931
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.165650
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.141775

Test set: Average loss: 0.0538, Accuracy: 9828/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.070990
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.108231
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.009608
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.026172
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.021797
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.072982
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.054367
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.034788
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.048998
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.085463
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.163623
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.020501
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.036627
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.014354
Train Epoch: 9 [

In [13]:
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [14]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size = 5)

        self.incept1 = InceptionA(in_channels = 10)
        self.incept2 = InceptionA(in_channels = 20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [19]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # Print
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [22]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [23]:
for epoch in range(1, 10):
    train(epoch)
    test()

<ipython-input-15-279515563701>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.103254
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.116769
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.238199
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.114147
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.090738
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.181097
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.221858
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.110610
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.214904
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.167566
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.161593
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.053505
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.191389
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.137297
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.223601
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.122190
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.174714
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.425176
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.141872
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.271640
Train Epoch: 1 [12800/60000 (

<ipython-input-22-5865b35a36fe>:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.1067, Accuracy: 9651/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.062259
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.085188
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.074310
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.117987
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.163016
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.181967
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.183993
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.069919
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.033011
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.184088
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.138950
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.136714
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.111739
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.136402
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.059585
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.028290
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.051708
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.081547
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.285404
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.006108
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.037795
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.043847
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.157007
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.053175
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.020865
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.089264
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.068739
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.019253
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.258010
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.022502
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.147717
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.103740
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.060768
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.051759
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.013265
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.034109
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.056871
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.032745
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.016152


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.047479
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.048072
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.051886
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.061879
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.105276
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.127174
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.030264
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.093879
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.010021
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.094528
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.042982
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.072581
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.030724
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.036938
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.008400
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.061050
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.019837
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.114707
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.078068
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.080745


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.050893
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.027448
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.006392
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.184859
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.061881
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.037134
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.039195
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.013795
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.033074
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.076878
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.041268
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.092975
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.005140
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.018288
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.010158
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.091525
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.044828
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.016863
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.097439
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.033882


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.005291
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.092259
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.056628
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.011230
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.055565

Test set: Average loss: 0.0602, Accuracy: 9822/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.039939
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.089644
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.045872
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.024956
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.012602
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.022027
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.167659
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.020983
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.027245
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.039116
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.010425
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.130464
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.016487
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.031674
Train Epoch: 9 [